In [1]:
import data_utils
train_data, test_data, user_num ,item_num, train_mat = data_utils.load_all()

In [2]:
print(train_mat.shape)

(6040, 3706)


In [3]:
print(item_num)

3706


In [4]:
print(user_num)

6040


In [6]:
print(train_data[-1])

[6039, 669]


In [9]:
print(train_mat[6039, 669])

1.0


In [7]:
# construct the train and test datasets
train_dataset = data_utils.NCFData(
		train_data, item_num, train_mat, 4, True)

In [8]:
train_dataset.ng_sample()


In [9]:
print(len(train_data))

994169


In [10]:
print(len(train_mat[1]))

128


In [11]:
print(len(train_data))

994169


In [12]:
print(len(test_data))

604000


In [13]:
import torch.utils.data as data
train_dataset = data_utils.NCFData(
		train_data, item_num, train_mat, 4, True)
train_loader = data.DataLoader(train_dataset,
		batch_size=1, shuffle=True, num_workers=1)

In [14]:
print(len(train_loader))

4970845


In [17]:
a = torch.randn(1, 3)
print(a)
print(torch.argmin(a))

tensor([[ 0.1533, -0.6597, -0.3431]])
tensor(1)


In [13]:
import torch
import torch.nn.functional as F
def poison_loss(users, fake_user, items, promoted_item, predictions, labels, _lambda, eita, kappa):
	
	# total_bce_loss = torch.sum(-1*labels * torch.log(predictions) - (1 - labels) * torch.log(1 - predictions))
	total_bce_loss = F.binary_cross_entropy(torch.sigmoid(predictions), labels, reduction='sum')
    # Getting the mean BCE loss
	mean_bce_loss = total_bce_loss

	# get predictions per user
	unique_users = torch.unique(users)
	users_loss = 0
	num_unique_users = 0
	for user in unique_users:
		
		# get predictions and corresponding items for each user
		indices = (users == user).nonzero().squeeze()
		user_predictions = torch.index_select(predictions, 0, indices)
		user_items = torch.index_select(items, 0, indices)
		user_labels = torch.index_select(labels, 0, indices)
		log_min_prediction = torch.log(torch.min(user_predictions)).item()
		
		# only consider users who have not rated promoted item yet
		indices = (user_items == promoted_item).nonzero()
		# print("indeces:",indices)
		if len(indices) != 0 and user_labels[indices[0].item()] > 0:
			continue
		
		if len((user_items==promoted_item).nonzero()) == 0:
			log_prob_tgt_item = 0
		else:
			log_prob_tgt_item = torch.log(user_predictions[(user_items==promoted_item).nonzero()[0].item()])
		# print(log_prob_tgt_item)
		user_loss = torch.max(torch.Tensor([log_min_prediction - log_prob_tgt_item, -1*kappa])).item()
		users_loss += user_loss
		num_unique_users += 1
	
	u = users == fake_user
	indices = u.nonzero().squeeze()
	fake_user_vector = torch.index_select(predictions, 0, indices)
	fake_user_l2_norm = torch.linalg.vector_norm(fake_user_vector)
	# print(fake_user_l2_norm)
	# print(users_loss)
	# print(mean_bce_loss)
	poison_loss = torch.pow(fake_user_l2_norm, 2) + eita*users_loss
	# print(poison_loss)

	return mean_bce_loss + _lambda*poison_loss

In [14]:
users = torch.Tensor([1,4,2,3,5,4,3,3,2,1,5,3,2,1,5,2,3,1,4,2,3,5,3,2,1,1,4,2,5,2,5,3])
items = torch.Tensor([4,1,6,2,5,8,9,4,0,4,1,3,4,5,2,6,7,9,8,7,0,4,6,2,3,1,5,0,9,8,5,3])
predictions = torch.Tensor([0.4,0.1,0.6,0.2,0.5,0.8,0.9,0.4,0,0.4,0.1,0.3,0.4,0.5,0.2,0.6,0.7,0.9,0.8,0.7,0.0,0.4,0.6,0.2,0.3,0.1,0.5,0,0.9,0.8,0.5,0.3])
labels = torch.Tensor([1,1,0,0,1,1,0,0,0,0,1,0,0,1,1,1,1,0,0,1,1,0,1,1,1,1,0,0,0,1,1,1])
fake_user = 2
promoted_item = 6

print(poison_loss(users, fake_user, items, promoted_item, predictions, labels, _lambda=0.01, eita=100, kappa=1))


tensor(18.9076)


In [5]:
import torch
t = torch.ones(5)*5
print(t)

tensor([5., 5., 5., 5., 5.])


In [12]:
indices = torch.tensor([1, 2])
t[indices] = t[indices] * 0.5
x = t*torch.Tensor([.5, .5, .5, .5, .5])
print(x)

tensor([2.5000, 0.1562, 0.1562, 2.5000, 2.5000])


In [13]:
for a, b in zip(x, t):
    print(a.item(), b.item())

2.5 5.0
0.15625 0.3125
0.15625 0.3125
2.5 5.0
2.5 5.0


In [3]:
x = [i for i in range(5)]
print(x)

[0, 1, 2, 3, 4]
